In [52]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq
import string
from nltk.corpus import stopwords 
import pybrain

from pybrain.tools.customxml.networkwriter import NetworkWriter
from pybrain.tools.customxml.networkreader import NetworkReader

In [8]:
def remove_punctuation(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    s = s.lower()
    return s
#TODO: fix this or dont use
def get_longword(s, n):
    s = re.split(' ', s)
    s =  [w for w in s if not w in stop]
    return heapq.nlargest(n, s, key=len)

def get_uniqwds(row, ans):
    answerwords  = []
    for col in row['answerA':'answerD']:
        col = re.split(' ', col)
        for c in col:
            answerwords.append(c)
    if ans == 'a':
        col = row['answerA']
    if ans == 'b':
        col = row['answerB']
    if ans == 'c':
        col = row['answerC']
    if ans == 'd':
        col = row['answerD']
    col = re.split(' ', col)
    uniq = [word for word in col if answerwords.count(word) == 1]
    return ' '.join(uniq)

def get_rarewrd(s):
    s = re.split(' ', s)
    s =  [w for w in s if not w in stop]
    freqs = [words.count(x) for x in s]
    dd= [i for i,x in enumerate(freqs) if x == min(freqs)]
    try:
        return [s[dd[0]]]
    except:
        return []

#TODO: make this work with one word/period
def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a
    
def convert_back2arrays(s):
    s = np.fromstring(s[3:-1], sep = ' ')
    return s
    
def run_through_theta(v):
    return v.dot(theta.as_matrix())

def get_winner_from_avecs(row):
	try:	
		dists = []
		for col in row['answerA':'answerD']:
		#TODO: if 'not' in words: 1-avec
			avec = get_avg_vec(col)
			#if 'not' in re.split(' ',col):
			#	avec = 1-avec
			#	print('NOT')
			#why nan errors?
			if sum(np.isfinite(avec)) < 300:
				print('avec')
				print(avec)
			if  sum(np.isfinite(row['pvec'])) < 300:
				print('pvec')
				print(row['pvec'])
			dist = cosine(avec, row['pvec'])
			dists.append(dist)
		m = np.nanmin(dists)
		best = [i for i, j in enumerate(dists) if j == m]
		if best == [0]:
			return 'A'
		if best == [1]:

			return 'B'
		if best == [2]:
			return 'C'
		if best == [3]:
			return 'D'
		else:
			print('returning C becuase no best cosine')
			print(len(best))
			print(dists)
			print(m)
			return 'C'
	except:
		print('some error, returning C')
		return 'C'
    
    
    
    
def word_clusters(row):
    if str(row.id)[-2:] == '01':
        print(row.id)
        print('\n\n\n\n')
    #print(row.correctAnswer)
    try:
        keywds = row.keyword

        word_pos = {}
        for keyw in keywds:
            ls = [i for i,x in enumerate(words) if x == keyw]
            #print(ls)
            word_pos[keyw] = ls

        spots = []
        for key1 in keywds:
            print(key1)
            matches = {}
            for z in word_pos[key1]:
                dists=[]
                for key_ in keywds:
                    d= abs(min(word_pos[key_], key=lambda x:abs(x-z))-z)
                    dists.append(d)
                #print(z)
                #print(dists)
                dists.remove(max(dists))
                matches[z] = sum(dists)

            zzz= [k for k, v in matches.items() if v ==min(matches.values())]
            spots.append(zzz[0])
        print(spots)
        answers = row['akeyword':'dkeyword']
        kkk = []
        for answer in answers:
            print(answer[0])
            waka = 0
            for x in spots:
                wd = words[x-500:x+500]
                ct = wd.count(answer[0])
                waka = (waka + ct)#/(words.count(answer[0])/2)    #trying to skew for 
            kkk.append(waka)
        mx = max(kkk)
        print(kkk)
        best = [i for i, j in enumerate(kkk) if j == mx]
        print(row['correctAnswer'])
        print(best)
        if len(best) > 1:
            best = best[0]
            
        if best == [0]:
            return 'A'
        if best == [1]:
            return 'B'
        if best == [2]:
            return 'C'
        if best == [3]:
            return 'D'
        else:
            print('returning C becuase no max')
            return 'C'
    except:
        print('returning C becuase of BIG PROBLEM')
        return 'C'

In [45]:
data  = pd.read_csv('../input/training_set_(add_cols).csv').head(500)
#data  = pd.read_csv('../input/validation_set.tsv', '\t').head(20)

In [46]:
data.qvec = data.qvec.apply(convert_back2arrays)
data.tvec = data.tvec.apply(convert_back2arrays)

In [47]:
isub = [len(data.tvec[i]) == 300 for i in data.index]
data = data.iloc[isub]
jsub = [len(data.qvec[i]) == 300 for i in data.index]
data = data.iloc[jsub]

In [51]:
qarray = np.array([data.qvec[i] for i in data.index])
tarray = np.array([data.tvec[i] for i in data.index])

In [58]:

from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer

In [56]:
net = buildNetwork(300, 3, 300)#, bias=True, hiddenclass=TanhLayer)

In [60]:
ds = SupervisedDataSet( 300, 300 )
ds.setField( 'input', qarray )
ds.setField( 'target', tarray )

In [61]:
trainer = BackpropTrainer( net, ds )

In [63]:
trainer.trainUntilConvergence( verbose = True, validationProportion = 0.15, maxEpochs = 10, continueEpochs = 10 )

('train-errors:', '[0.001508 , 0.001508 , 0.001507 , 0.001507 , 0.001507 , 0.001507 , 0.001507 , 0.001507 , 0.001507 , 0.001507 , 0.001507 ]')
('valid-errors:', '[0.00151  , 0.001513 , 0.001515 , 0.001512 , 0.001516 , 0.001514 , 0.001513 , 0.001514 , 0.001515 , 0.001513 , 0.001514 , 0.001514 ]')


([0.001507831285769674, 0.0015077699612289558],
 [0.0015096119700622644, 0.0015134325831781749, 0.0015148175877987679])

In [69]:
p = net.activateOnDataset( ds )

In [70]:
p

array([[  4.22996679e-02,   1.93372321e-02,  -1.34562143e-03, ...,
         -1.02419169e-02,   2.23898740e-02,   1.89055341e-03],
       [  3.97357561e-02,   1.68923851e-02,   2.89194796e-03, ...,
         -1.28711221e-02,   2.19910508e-02,   5.37066698e-03],
       [  4.15442418e-02,   1.88502804e-02,   7.57670466e-04, ...,
         -1.10275681e-02,   2.17691655e-02,   2.98627008e-03],
       ..., 
       [  4.17517699e-02,   1.53594133e-02,  -1.18240140e-02, ...,
         -9.84643494e-03,   2.86891093e-02,   7.60787437e-05],
       [  4.18364904e-02,   1.95300474e-02,  -1.99825549e-03, ...,
         -1.30872830e-02,   2.40876678e-02,   7.17772012e-03],
       [  4.15003542e-02,   1.84983274e-02,   8.48186519e-04, ...,
         -1.03363550e-02,   2.14634946e-02,   1.57505657e-03]])

In [72]:
data.qvec[1]

array([  2.17765472e-02,   2.55097200e-02,   8.36485056e-03,
         1.52464695e-03,  -4.60739157e-02,   1.15439016e-02,
         2.50939138e-02,  -3.79874974e-02,   2.12105153e-02,
         5.71956784e-03,   2.63803620e-03,  -4.38174410e-02,
         4.08548322e-03,   2.61384159e-02,  -4.89751939e-02,
         9.76818666e-02,  -5.11240400e-03,   6.72472012e-02,
        -4.03809248e-02,  -3.83159170e-02,   3.06978211e-02,
         5.61112165e-02,  -1.22637741e-02,   1.03424205e-02,
         1.18597551e-02,  -1.72664211e-02,  -3.36079774e-03,
         3.47723313e-02,  -1.48003906e-02,   3.78846927e-02,
         1.41722044e-02,  -3.12777594e-02,  -4.19230291e-03,
         3.75995787e-02,   4.21836235e-02,   2.49174107e-02,
        -4.28466722e-03,   4.42558862e-03,   2.21808654e-02,
        -7.77546614e-03,   4.78679992e-02,  -1.24259923e-02,
        -1.74375981e-02,  -3.34478196e-02,  -1.90005998e-02,
        -3.43516767e-03,  -5.24764336e-02,   6.59037307e-03,
        -3.76760513e-02,

In [71]:
net.activate(data.qvec)

array([ 0.03973576,  0.01689239,  0.00289195,  0.01325472, -0.04295635,
       -0.0095751 ,  0.00466456, -0.03651943,  0.01953941,  0.03047077,
       -0.01456932, -0.02762214,  0.00035726,  0.00570665, -0.0387442 ,
        0.02665012, -0.02651955,  0.04412014, -0.0013718 , -0.01746958,
       -0.00424636,  0.00300367, -0.01694011, -0.00555841, -0.01710689,
       -0.03388686, -0.02855031,  0.01078027, -0.0001721 , -0.02962741,
       -0.02317643, -0.02930378,  0.00212708,  0.00198238, -0.02123117,
       -0.01482073,  0.00472835, -0.00060479,  0.01079994,  0.02767823,
        0.02061899, -0.00891803,  0.01231008,  0.00541759, -0.00745864,
       -0.04760679, -0.00939598,  0.00742278, -0.0169771 ,  0.00280643,
       -0.0028462 , -0.02109291, -0.01609308, -0.02748478,  0.01080286,
        0.01044021, -0.01128519, -0.03518641,  0.00533438, -0.01682758,
       -0.0136268 ,  0.00976111, -0.02364817, -0.02385195, -0.00327636,
       -0.00771024, -0.01687083,  0.03670945, -0.00475691,  0.01

In [57]:
net

<FeedForwardNetwork 'FeedForwardNetwork-8'>

In [10]:
data.head()

,Unnamed: 0,id,question,correctAnswer,answerA,answerB,answerC,answerD,keyword,qvec,akeyword,bkeyword,ckeyword,dkeyword,aavec,bavec,cavec,davec,tvec
0,0,100001,When athletes begin to exercise their heart ra...,C,at the tissue level,at the organ level,at the system level,at the cellular level,"['organization', 'respiration', 'coordinate', ...","[0.0110091, -0.01009273, 0.01186427, 0.0108366...",['tissue'],['organ'],['system'],['cellular'],[-0.06857704 0.0755747 -0.02295232 -0.033028...,[ 0.03915285 0.06828986 0.01987994 -0.039456...,[ -8.58545711e-04 -6.56884313e-02 3.6975562...,[-0.04597223 -0.04894885 -0.02629347 -0.015296...,[ -8.58545711e-04 -6.56884313e-02 3.6975562...
1,1,100002,Which example describes a learned behavior in ...,C,smelling the air for odors,barking when disturbed,sitting on command,digging in soil,"['describes', 'behavior', 'example', 'learned'...","[0.0217765472, 0.02550972, 0.00836485056, 0.00...",['smelling'],['disturbed'],['sitting'],['digging'],[-0.01314746 0.06018303 0.02193587 -0.030513...,[ -1.91806387e-02 6.23145625e-02 4.6825971...,[ 8.58191550e-02 4.23244461e-02 3.1987138...,[-0.0030783 0.0894921 -0.00188119 0.072752...,[ 8.58191550e-02 4.23244461e-02 3.1987138...
2,2,100003,When two nuclei are combined into one nucleus ...,D,conversion,reaction,fission,fusion,"['combined', 'nucleus', 'release', 'process', ...","[0.0108336833, 0.033334874, 0.0231488449, -0.0...",['conversion'],['reaction'],['fission'],['fusion'],[ 5.07117175e-02 4.24134396e-02 -3.9954688...,[ 2.67807711e-02 -4.42314669e-02 2.1424615...,[ -1.63884610e-02 1.10206462e-01 5.7715885...,[ -2.58576833e-02 5.64112153e-04 1.1058631...,[ -2.58576833e-02 5.64112153e-04 1.1058631...
3,3,100004,Which is a distinction between an epidemic and...,B,the symptoms of the disease,the geographical area affected,the species of organisms infected,the season in which the disease spreads,"['distinction', 'epidemic', 'pandemic', 'betwe...","[0.0247666957, -0.0448009823, 0.0364875012, 0....",['symptoms'],['geographical'],['organisms'],['disease'],[ -3.53830308e-03 8.31341818e-02 -1.4726990...,[ 0.00751284 0.02615282 0.00507625 0.050031...,[ 2.58413237e-03 7.08790570e-02 2.3241139...,[ -1.82848312e-02 3.75033580e-02 -5.4465457...,[ 0.00751284 0.02615282 0.00507625 0.050031...
4,4,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...,"['different', 'which', 'orbit', 'comet', 'orbit']","[0.0375164593, 0.0500212982, -0.00442899968, 0...",['circular'],['elliptical'],['orbital'],['predictable'],[-0.02845142 0.01472486 0.04459053 0.032610...,[ 1.61254369e-02 -3.62438373e-02 2.1020658...,[-0.06340646 -0.0265039 -0.03779025 0.035761...,[ 2.54285671e-02 -2.26645917e-02 -7.1863338...,[ 1.61254369e-02 -3.62438373e-02 2.1020658...
